In [1]:
import torch
import torch.nn as nn
from collections import defaultdict
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import tqdm
import math
from collections import defaultdict

In [2]:
FILL_IN = "FILL_IN"

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

names = []
names_path = '/content/drive/MyDrive/ADL_HW/names.txt'
with open(names_path, 'r') as f:
    names = f.read().splitlines()

Mounted at /content/drive/


In [4]:
names

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn',
 'abigail',
 'emily',
 'elizabeth',
 'mila',
 'ella',
 'avery',
 'sofia',
 'camila',
 'aria',
 'scarlett',
 'victoria',
 'madison',
 'luna',
 'grace',
 'chloe',
 'penelope',
 'layla',
 'riley',
 'zoey',
 'nora',
 'lily',
 'eleanor',
 'hannah',
 'lillian',
 'addison',
 'aubrey',
 'ellie',
 'stella',
 'natalie',
 'zoe',
 'leah',
 'hazel',
 'violet',
 'aurora',
 'savannah',
 'audrey',
 'brooklyn',
 'bella',
 'claire',
 'skylar',
 'lucy',
 'paisley',
 'everly',
 'anna',
 'caroline',
 'nova',
 'genesis',
 'emilia',
 'kennedy',
 'samantha',
 'maya',
 'willow',
 'kinsley',
 'naomi',
 'aaliyah',
 'elena',
 'sarah',
 'ariana',
 'allison',
 'gabriella',
 'alice',
 'madelyn',
 'cora',
 'ruby',
 'eva',
 'serenity',
 'autumn',
 'adeline',
 'hailey',
 'gianna',
 'valentina',
 'isla',
 'eliana',
 'quinn',
 'nevaeh',
 'ivy',
 'sadie',
 'piper',
 'lydia',
 'alexa',
 'josephine',
 'emery',
 'julia'

In [5]:
# Dictionaries, {idx -> ch} and {ch -> idx}
itos = defaultdict(int)
stoi = defaultdict(int)
# Number of characters used to predict the target character in the MLP Language Model
block_size = 3
# Batch size used in MLP Language Model
batch_size = 32
# Embedding dimension, per character
d_model = 10
# Hidden dimension for RNN and also MLP Language Models
d_h = 200

# START = START token
stoi['.'] = 0
itos[0] = '.'

count = 1
# Loop over all names and create mappings itos and stoi mapping a unique character to a idx
for name in names:
  for ch in name:
    if not ch in stoi:
      stoi[ch] = count
      itos[count] = ch
      count = count + 1

In [6]:
assert len(stoi) == len(itos)
vocab_size = len(stoi)
assert vocab_size == 27

In [7]:
stoi

defaultdict(int,
            {'.': 0,
             'e': 1,
             'm': 2,
             'a': 3,
             'o': 4,
             'l': 5,
             'i': 6,
             'v': 7,
             's': 8,
             'b': 9,
             'p': 10,
             'h': 11,
             'c': 12,
             'r': 13,
             't': 14,
             'y': 15,
             'n': 16,
             'g': 17,
             'z': 18,
             'f': 19,
             'd': 20,
             'u': 21,
             'k': 22,
             'w': 23,
             'q': 24,
             'x': 25,
             'j': 26})

## BiGram Language Model
- Implement the Bigram Language Model
- Get all the relevent counts, then get the train dataset Perplexity
- Use the class notes

In [14]:
# Using the formulas in class, loop over each name and get the parameters
c1 = defaultdict(int)
c2 = defaultdict(int)
c2_sum = defaultdict(int)
for name in open(names_path, 'r'):
    # Lowercase and remove any whitespace at the end
    name = name.lower().strip()
    # Pad with START = '.' and STOP = '.'
    name = '.'+name+'.'
    # Transform to integer
    name = [stoi[ch] for ch in name]
    # Get the counts for Bigrams and Unigrams
    for i in range(len(name)):
      c1[name[i]] += 1

      if i < len(name) - 1:
        bigram = (name[i], name[i + 1])
        c2[bigram] += 1
        c2_sum[name[i]] += 1

In [15]:
# Get perplexity

sumneglogp = 0
T = 0
for name in open(names_path, 'r'):
    # Get rid of white space and lowercase
    name = name.lower().strip()
    # Get the length of the word, without padding
    T += len(name)
    # Don't pad the STOP since we are not generating; pad with START
    name = '.' + name
    # Transform to integrs
    name = [stoi[ch] for ch in name]
    # Get the loss -log(p(name)); use that the log of the product is the sum of the logs
    for i in range(len(name) - 1):
      x, y = name[i], name[i + 1]
      p = c2[(x, y)] / c1[x]

      sumneglogp += -math.log(p)
# Print the Perplexity
print('Preplexity: ', torch.pow(2.0, torch.tensor(sumneglogp / T)).item())

Preplexity:  6.484745025634766


In [16]:
# Generate a random word using this distributon
# When you generate STOP, terminate
name = '.'
while True:
    c = stoi[name[-1]]
    # Make the distribution from c to any other word other than START
    p = []
    for d in range(vocab_size):
        # Use the same indicies as the dictionary we set up
        # Populate p
        p.append(c2[(c,d)]/c1[c])
    #print(p)
    assert len(p) == vocab_size
    # Sample randmly from the probability using torch.Categorical
    c = torch.distributions.Categorical(torch.tensor(p)).sample()
    # Offset by 1 since we want indices [1, 2, ..., vocab_size]
    if c.item() == 0:
        break
    else:
        name += itos[c.item()]
print('Generated name: ' , name[1:])


Generated name:  mujueldeyn


## MLP Language Model

- Implement the MLP language model from below
- Look at page 7, Equation (1)
- https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf

In [17]:
x_data = []
y_data = []
for name in open(names_path, 'r'):
    name = name.lower().strip()
    # Pad with block_size START tokens and 1 STOP token
    name = ''.join(block_size * ['.']) + name + '.'
    # Loop through name and get the (x, y) pairs
    # Add (x, y) to x_data and y_data and make sure you transform to characters
    # Make sure x_data and y_data have integers, use stoi
    for i in range(len(name) - block_size):
      x = name[i: i + block_size]
      y = name[i + block_size]
      x_data.append([stoi[ch] for ch in x])
      y_data.append(stoi[y])

In [18]:
class MLPLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # An embedding for each character; vocab_size of them
        self.e = nn.Embedding(vocab_size, d_model)
        # H; should take in block_size * d_model vector and output d_h
        self.H = nn.Linear(block_size * d_model, d_h)
        # U; should take in d_h vector and output vocab_size
        self.U = nn.Linear(d_h, vocab_size)
        # W; for the skip connection, should take in block_size * d_model and output vocab_size
        self.W = nn.Linear(block_size * d_model, vocab_size)

    # x should be (batch_size, block_size)
    def forward(self, x):
        x = self.e(x)
        x_flat = x.view(x.shape[0], -1)
        hid = torch.relu(self.H(x_flat))
        logits = self.U(hid) + self.W(x_flat)
        return logits

In [19]:
x_data[0], y_data[0]

([0, 0, 0], 1)

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [21]:
# Define a dataloader with x_data and y_data with batch_size
dl = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(
        torch.tensor(x_data, dtype=torch.long),
        torch.tensor(y_data, dtype=torch.long)
    ),
    batch_size=batch_size,
    shuffle=True
)

In [22]:
for xb, yb in dl:
    assert xb.shape == (batch_size, 3)
    assert yb.shape == (batch_size,)
    break

In [23]:
from torch import optim
# Define the MLP model and the Adam optimizer learning rate 0.001
model = MLPLanguageModel().to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
total_loss = 0
total_ct = 0
total_epochs = 20

loss_fn = torch.nn.CrossEntropyLoss().to(device)

for _ in range(total_epochs):
    for xb, yb in dl:
        xb = xb.to(device)
        yb = yb.to(device)
        # Zero the gradients
        optimizer.zero_grad()

        # Get the logits
        logits = model(xb)

        # Compute the loss
        loss = loss_fn(logits, yb)

        # Get the new gradient
        loss.backward()

        # Clip the gradients to max norm 0.1
        # Use clid_grad_norm from torch
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)

        # Do a gradient update
        optimizer.step()

        # Get the loss for the batch and get the number of batches
        total_loss += loss.item()
        total_ct += 1

        # Print the loss
        if total_ct and total_ct % 500 == 0:
            print(total_loss / total_ct)
            total_loss = 0
            total_ct = 0

In [ ]:
with torch.no_grad():
    # Get perplexity
    sumneglogp = 0
    T = 0
    for name in open(names_path, 'r'):
        name = name.lower().strip()
        T += len(name)
        # Pad with block_size START tokens
        name = ''.join(['.'] * block_size) + name
        x_data = []
        y_data = []
        # Gather all the terms over the loss
        # Notice that we compute -log p(...abc)
        # Which is -log p(a | ...) - log p(b | a..) - log p(c | ba.)
        for i in range(len(name) - block_size):
            x = name[i:i+block_size]
            y = name[i+block_size]
            x_data.append([stoi[ch] for ch in x])
            y_data.append(stoi[y])
        # Gather the loss over the name, for each term
        # You need to get the softmax loss for each term
        # Can either use the CrossEntropyLoss or do this manually
        # Compute the loss
        xb = torch.tensor(x_data, dtype=torch.long).to(device)
        yb = torch.tensor(y_data, dtype=torch.long).to(device)
        logits = model(xb)

        # Use reduction "sum" so you don't need to worry about N
        loss = torch.nn.CrossEntropyLoss(reduction="sum")(logits, yb)

        # Change to log base 2
        loss *= (1.0 / torch.log(torch.tensor(2.0)))

        sumneglogp += loss

    print('Preplexity: ', torch.pow(2.0, sumneglogp.clone().detach() / T).item())

In [ ]:
# Generate a random word using this distributon
# When you generate STOP, terminate
name = ''.join(block_size * ['.'])
while True:
    # Get the idx
    c = torch.tensor([stoi[ch] for ch in name[-block_size:]], dtype=torch.long).unsqueeze(0).to(device)
    # Make the distribution from c to any other word other than START
    logits = model(c)
    p = torch.softmax(logits, dim=1)
    # Randomly sample from p a new character
    c = torch.distributions.Categorical(p).sample()
    if c.item() == 0:
        break
    else:
        name += itos[c.item()]
print('Generated name: ' , name[block_size:])

## RNN Language Model
- For each name, run an RNN character by character
- Use the recursion x = Tanh()(Wh @ h + Wx @ x + bh + bx) and y = Softmax()(Wy h + by)
- Do not use the RNN Cell from PyTorch, do this manually as hinted below

In [26]:
class RNNLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Each token has an embedding of size vocab_size
        self.e = nn.Embedding(vocab_size, d_model)
        # Wh used to map hidden to hidden
        self.Wh = nn.Linear(d_model, d_model, bias = False)
        self.Wx = nn.Linear(d_model, d_model, bias = False)
        self.Wy = nn.Linear(d_model, vocab_size, bias = False)

    def forward(self, x, h):
        # Run through to get the embedding for the token
        # The embedding per token is the feature vector x  we pass into the
        # Represent x as an embedding
        x = self.e(x)
        # Get the hidden state
        h = torch.tanh(self.Wx(x) + self.Wh(h))
        # Get the logits we use to predict y
        z = self.Wy(h)
        # Return the z predicting y for the timestep we are at and the next hidden state
        return z, h

In [27]:
model = RNNLanguageModel()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [28]:
total_loss = 0
total_ct = 0
total_epochs = 5

for _ in range(total_epochs):
    for name in open(names_path, 'r'):
        name = name.lower().strip()
        # Add the start and end padding token
        name = '.' + name + '.'
        # name[:-1]
        x_data = torch.tensor([stoi[c] for c in name[:-1]], dtype=torch.long)
        # name[1:]
        y_data = torch.tensor([stoi[c] for c in name[1:]], dtype=torch.long)
        logits = []
        # Set the hidden state to random
        h = torch.zeros(1, d_model)
        # Zero the grad
        optimizer.zero_grad()

        # Loop through each token and get the new h and then pass it forward
        # Accumulate all the logits
        for x in x_data:
            z, h = model(x.unsqueeze(0), h)

            logits.append(z)

        # Put all the logits into one tensor
        logits = torch.cat(logits, dim=0)

        # Compute the loss
        loss = F.cross_entropy(logits, y_data)

        # Get the new gradient
        loss.backward()

        # Clip the gradients at max norm 0.1
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)

        # Do a gradient update
        optimizer.step()

        # Get the loss for the batch and get the number of batches
        total_loss += loss.item()
        total_ct += 1

        if total_ct and total_ct % 100 == 0:
            print(total_loss / total_ct)
            total_loss = 0
            total_ct = 0


3.403815312385559
3.39144540309906
3.370520541667938
3.3381054639816283
3.3522675371170045
3.3422869634628296
3.3254690861701963
3.3027714705467224
3.303741476535797
3.3092859864234923
3.281665642261505
3.2837720251083375
3.2938400173187254
3.2847199559211733
3.262160098552704
3.252235553264618
3.239172296524048
3.226921045780182
3.217320775985718
3.211389660835266
3.215096845626831
3.183451974391937
3.1514693546295165
3.166835379600525
3.1571210289001463
3.1387170219421385
3.088104884624481
3.0608856511116027
3.0885117864608764
3.042241072654724
2.9829265785217287
2.994392685890198
2.9893229746818544
2.9392813444137573
2.932671754360199
2.854076840877533
2.867024531364441
2.8692841863632204
2.8367901420593262
2.7989142179489135
2.8062514567375185
2.823779683113098
2.764306285381317
2.7770681285858156
2.7531023693084715
2.7305845618247986
2.7070208859443663
2.7169364404678347
2.6647800254821776
2.696675441265106
2.693651967048645
2.6939841079711915
2.672834002971649
2.697805895805359
2

KeyboardInterrupt: 

In [33]:
with torch.no_grad():
    # Get perplexity
    sumneglogp = 0
    T = 0
    for name in open(names_path, 'r'):
        name = name.lower().strip()
        T += len(name)
        name = '.' + name
        # Get the name from index 0 to -1 exclusive end
        x_data = torch.tensor([stoi[c] for c in name[:-1]], dtype=torch.long)
        # Get the y from index 1 to end inclusive end
        y_data = torch.tensor([stoi[c] for c in name[1:]], dtype=torch.long)
        # logits per token prediction
        logits = []
        # Initialize the h vector to random
        h = torch.zeros(1, d_model)
        # Loop over each chracter in the name and pass h and this into the RNN
        # Get the new logit
        for x in x_data:
            # Get the int for x
            x = x.unsqueeze(0)
            # Get z and h
            z, h = model(x, h)
            # Append to logit
            logits.append(z)

        # Get all the logits for each character
        logits = torch.cat(logits, dim=0)

        # Compute the loss across all characters
        loss = F.cross_entropy(logits, y_data, reduction='sum')

        # Change to log base 2
        # log2(x) = ln(x) / ln(2)
        loss /= math.log(2)

        sumneglogp += loss.item()

    # sumneglogp is -log(p('.' + name1)) -log(p('.' + name2)) -log(p('.' + name3)) ...
    # Divide by the appropriate term to get the answer we want
    print('Preplexity: ', torch.pow(2, torch.tensor(sumneglogp / T)).item())

Preplexity:  14.54911994934082


In [45]:
# Generate a random word using this distributon
# Intialize the word with
name = '.'
# Initialize h to random
h = torch.zeros(1, d_model)
while True:
    # Make c to an integer
    c = torch.tensor([stoi[name[-1]]], dtype=torch.long)
    # Make the distribution from c to any other word other than START
    logits, h = model(c, h)
    # Get p; use Softmax
    p = torch.softmax(logits, dim=1)
    # Sample from p
    c = torch.multinomial(p, num_samples=1)
    # If we generate '.', stop
    if c.item() == 0:
        break
    else:
        name += itos[c.item()]
print('Generated name: ' , name[1:])

Generated name:  ceyro
